In [20]:
import torch.nn as nn
import torch
from PIL import Image
import numpy as np
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import os

In [21]:
# Step 1
# Get the image and change every image to an 256-dimention vector
dataSet = np.zeros([10, 256])
for i in range(0, 10):
    inputImageDir = './input/' + str(i) + '.png'
    inputImage = Image.open(inputImageDir)
    inputImage = inputImage.convert("1")
    inputImage.save(inputImageDir)
    data = inputImage.getdata()
    array = np.array(data)/255
    dataSet[i] = array
dataSet = np.array(dataSet)

In [22]:
# Step 2
# define a neural network
class Perceptron(nn.Module):

    def __init__(self, input_size, num_hidden, num_classes):
        super(Perceptron, self).__init__()
        self.inputlayer = nn.Linear(input_size, num_hidden)
        self.hiddenlayer = nn.Linear(num_hidden, num_hidden)
        self.outputlayer = nn.Linear(num_hidden, num_classes)
        self.activate = nn.Sigmoid()

    def forward(self, x):
        res = self.inputlayer(x)
        res = self.activate(res)
        res = self.hiddenlayer(x)
        res = self.activate(res)
        res = self.outputlayer(x)
        res = self.activate(res)
        return res

In [23]:
class DigitDataset(Dataset):
    def __init__(self, dataset, label_list):
        self.dataset = dataset
        self.label_list = label_list
    def __len__(self):
        return len(self.label_list)
    def __getitem__(self, idx):
        data = self.dataset[idx]
        label = self.label_list[idx]
        return {
            'data': torch.from_numpy(data).float(),
            'label': torch.from_numpy(label).float()
        }

In [24]:
# Parameters of training
input_size = 256
num_classes = 256
num_hidden = 256
learning_rate = 0.001
batch_size = 10
num_epochs = 600

# Load the dataset
train_dataset = DigitDataset(dataset = dataSet, label_list = dataSet)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)
device = torch.device('cpu')
model = Perceptron(input_size=input_size, num_hidden=num_hidden, num_classes=num_classes).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [25]:
# from transformers import AutoModelForSequenceClassification
# Step 3
def train(dataloader, model, num_epochs):
    for epoch in range(num_epochs):
        losses = []
        ERROR_Train = [] 
        model.train() 
        for i, data in enumerate(dataloader, 0):
            model.zero_grad()
            real_cpu, label_cpu = data['data'], data['label']
            if torch.cuda.is_available():
                real_cpu = real_cpu.cuda() 
                label_cpu = label_cpu.cuda()
            real = real_cpu
            label = label_cpu
            inputv = Variable(real)
            labelv = Variable(label)
            output = model(inputv)
            err = criterion(output, labelv) 
            err.backward() 
            optimizer.step() 

            losses.append(err.data.item())
            error = label - output.data
#             print(error.shape)
            ERROR_Train.extend(error)
#         print(ERROR_Train)
        MAE = torch.mean(torch.abs(torch.stack(ERROR_Train)))
        ME = torch.mean(torch.stack(ERROR_Train))
        STD = torch.std(torch.stack(ERROR_Train)) 
        if epoch % 10 == 0 or epoch == num_epochs - 1:
            print('[%d/%d] Loss: %.4f MAE: %.4f Mean Error: %.4f STD: %.4f' % (epoch, num_epochs, np.average(losses), MAE, ME, STD))
    return output

# Start training      
output = train(train_loader, model, num_epochs)
# model = AutoModelForSequenceClassification.from_pretrained(output_dir).to(device)
# print(output.type)

[0/600] Loss: 0.2575 MAE: 0.4892 Mean Error: 0.4034 STD: 0.3079
[10/600] Loss: 0.0439 MAE: 0.1558 Mean Error: 0.0956 STD: 0.1864
[20/600] Loss: 0.0309 MAE: 0.0909 Mean Error: 0.0334 STD: 0.1726
[30/600] Loss: 0.0282 MAE: 0.0780 Mean Error: 0.0196 STD: 0.1669
[40/600] Loss: 0.0265 MAE: 0.0731 Mean Error: 0.0163 STD: 0.1621
[50/600] Loss: 0.0250 MAE: 0.0704 Mean Error: 0.0153 STD: 0.1575
[60/600] Loss: 0.0236 MAE: 0.0687 Mean Error: 0.0142 STD: 0.1531
[70/600] Loss: 0.0223 MAE: 0.0671 Mean Error: 0.0134 STD: 0.1489
[80/600] Loss: 0.0211 MAE: 0.0657 Mean Error: 0.0128 STD: 0.1448
[90/600] Loss: 0.0200 MAE: 0.0640 Mean Error: 0.0123 STD: 0.1408
[100/600] Loss: 0.0189 MAE: 0.0624 Mean Error: 0.0120 STD: 0.1369
[110/600] Loss: 0.0179 MAE: 0.0607 Mean Error: 0.0118 STD: 0.1331
[120/600] Loss: 0.0169 MAE: 0.0592 Mean Error: 0.0114 STD: 0.1295
[130/600] Loss: 0.0160 MAE: 0.0576 Mean Error: 0.0111 STD: 0.1260
[140/600] Loss: 0.0151 MAE: 0.0561 Mean Error: 0.0108 STD: 0.1226
[150/600] Loss: 0.014

In [ ]:
# Step 4
# Step 4a
# Export the image after training
# Before executing this block, create a folder called "output"
if not os.path.exists('./output'):
    os.mkdir('./output')
output_np = output.cpu().detach().numpy()
print(output_np.shape)
torch.save(model, 'net.pkl')
output_dataset = np.zeros([10, 256])
for i in range(10):
    output_img = output_np[i].reshape(16, 16)*255
    img = Image.fromarray(np.uint8(output_img))
    img = img.convert("1")
    output_path = './output/' + str(i) + '.png'
    img.save(output_path)
    data = img.getdata()
    array = np.array(data)/255
    output_dataset[i] = array

In [ ]:
# Step 4b
# Calculate Fh
def calculateFh(input_dataset, output_dataset):
    x, y = input_dataset.shape
    Fh_denominator = 0    # Fh分母
    Fh_numerator = 0      # Fh分子
    Fh_array = np.zeros([x])
    for j in range(x):
        for i in range(y):
            if input_dataset[j][i] == 0:
                Fh_denominator = Fh_denominator + 1
                if output_dataset[j][i] == 0:
                    Fh_numerator = Fh_numerator + 1
        Fh = Fh_numerator / Fh_denominator
        Fh_array[j] = Fh
    return Fh_array

In [ ]:
# Calculate Ffa
def calculateFfa(input_dataset, output_dataset):
    x, y = input_dataset.shape
    Ffa_denominator = 0    # Ffa分母
    Ffa_numerator = 0      # Ffa分子
    Ffa_array = np.zeros([x])
    for j in range(x):
        for i in range(y):
            if input_dataset[j][i] == 1:
                Ffa_denominator = Ffa_denominator + 1
            if output_dataset[j][i] == 0 and input_dataset[j][i] == 1:
                Ffa_numerator = Ffa_numerator + 1
        Ffa = Ffa_numerator / Ffa_denominator
        Ffa_array[j] = Ffa
    return Ffa_array

In [ ]:
Fh_array = calculateFh(dataSet, output_dataset)
Ffa_array = calculateFfa(dataSet, output_dataset)
print(Fh_array)
print(Ffa_array)

In [ ]:
# TODO: Step 4c: Graph Fh as a function of Ffa for each exemplar in the input dataset

In [57]:
# Step 5
def gaussian_noise(img, mean, sigma):
    # Generate gauss noise
    noise = np.random.normal(mean, sigma, img.shape)
    # Add the noise to image
    gaussian_out = img + noise
    # Make the value between 0 and 1
    gaussian_out = np.clip(gaussian_out, 0, 1)
    return gaussian_out

gaussian_dataset = np.zeros([9, 10, 256])
std = [0.001, 0.002, 0.003, 0.005, 0.01, 0.02, 0.03, 0.05, 0.1]
if not os.path.exists('./input_noise/'):
    os.mkdir('./input_noise/')
for j in range(9):
    if not os.path.exists('./input_noise/' + str(std[j])):
        os.mkdir('./input_noise/' + str(std[j]))
    for i in range(10):
        inputImage = dataSet[i]
        gaussian_data = gaussian_noise(inputImage, 0, std[j])
        img = gaussian_data.reshape(16, 16)*255
        img = Image.fromarray(np.uint8(img))
        img.convert("1")
        inputImageDir = './input_noise/' + str(std[j]) + '/' + str(i) + '.png'
        img.save(inputImageDir)
        gaussian_dataset[j][i] = gaussian_data
gaussian_dataset = np.array(gaussian_dataset)


Fh_noise_array = np.zeros([9, 10])
Ffa_noise_array = np.zeros([9, 10])

# Train 9 datasets with noise
if not os.path.exists('./output_noise/'):
    os.mkdir('./output_noise/')
for j in range(9):
    train_noise_dataset = DigitDataset(dataset = gaussian_dataset[j], label_list = gaussian_dataset[j])
    train_noise_loader = DataLoader(dataset=train_noise_dataset, batch_size=batch_size, shuffle=False)
#     print('Training dataset with noise standard deviation ' + str(std[j]))
    model_noise = torch.load('net.pkl') #  Load the model that trained before
#     output_noise = train(train_noise_loader, model_noise, num_epochs)   # Train
    output_noise = model_noise(torch.from_numpy(gaussian_dataset[j]).float()) # Use the model trained before to test
#     print('------------------------------------')
    output_noise_np = output_noise.cpu().detach().numpy()     # Get the output
    output_noise_dataset = np.zeros([10, 256])
#     Make the output only has 0 or 1
    if not os.path.exists('./output_noise/' + str(std[j])):
        os.mkdir('./output_noise/' + str(std[j]))
    for i in range(10):
        output_noise_img = output_noise_np[i].reshape(16, 16)*255
        img = Image.fromarray(np.uint8(output_noise_img))
        img = img.convert("1")
        output_path = './output_noise/' + str(std[j]) + '/' + str(i) + '.png'
        img.save(output_path)
        data = img.getdata()
        array = np.array(data)/255
        output_noise_dataset[i] = array
#     Calculate Fh and Ffa
    Fh = calculateFh(gaussian_dataset[j], output_noise_dataset)
    Ffa = calculateFfa(gaussian_dataset[j], output_noise_dataset)
    Fh_noise_array[j] = Fh
    Ffa_noise_array[j] = Ffa
print('------------Fh_noise_array------------')
print(Fh_noise_array)
print('------------Ffa_noise_array------------')
print(Ffa_noise_array)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)

In [ ]:
# TODO: Step 6: Display Data from your Tests in Step 5